A simple mean reversion strategy that uses the stationarity test on AAPL stock data from Yahoo Finance to identify periods of deviation from the mean and take positions accordingly python and yahoo finacne

In [5]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Retrieve AAPL stock data from Yahoo Finance
aapl = yf.download('AAPL', start='2010-01-01', end='2023-05-09')
def adf_test(series):
    """Perform ADF test on a time series and return p-value"""
    result = adfuller(series)
    pvalue = result[1]
    return pvalue

# Define window size for SMA
window = 20

# Compute SMA and standard deviation
aapl['SMA'] = aapl['Adj Close'].rolling(window=window).mean()
aapl['Std'] = aapl['Adj Close'].rolling(window=window).std()

# Compute upper and lower bands for mean reversion strategy
aapl['Upper'] = aapl['SMA'] + aapl['Std']
aapl['Lower'] = aapl['SMA'] - aapl['Std']

# Initialize position and cash balance
position = 0
cash = 100000

# Define trading function
def trade(date, price):
    global position, cash
    if position == 0 and price < aapl.loc[date, 'Lower']:
        # Enter long position if price falls below lower band
        position = cash / price
        cash = 0
    elif position > 0 and price > aapl.loc[date, 'SMA']:
        # Exit long position if price rises back above SMA
        cash = position * price
        position = 0

# Loop over time series and perform trades
for date, row in aapl.iterrows():
    if date >= aapl.index[0] + pd.Timedelta(days=window):
        # Compute p-value of ADF test for the past 20 days
        series = aapl.loc[date-pd.Timedelta(days=window):date, 'Adj Close']
        pvalue = adf_test(series)

        # Perform trade if p-value is below 0.05
        if pvalue < 0.05:
            trade(date, row['Adj Close'])





[*********************100%***********************]  1 of 1 completed


A simple mean reversion strategy that uses stationarity test such as ADF and includes half life on AAPL stock using Yahoo Finance data python and yahoo finance

In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

# Retrieve AAPL stock data from Yahoo Finance
aapl = yf.download('AAPL', start='2010-01-01', end='2023-05-09')

# Define window size for SMA
window = 20

# Compute SMA and standard deviation
aapl['SMA'] = aapl['Adj Close'].rolling(window=window).mean()
aapl['Std'] = aapl['Adj Close'].rolling(window=window).std()

# Compute upper and lower bands for mean reversion strategy
aapl['Upper'] = aapl['SMA'] + aapl['Std']
aapl['Lower'] = aapl['SMA'] - aapl['Std']

# Compute half-life of mean reversion using ADF test
adf_result = adfuller(aapl['Adj Close'])
halflife = -np.log(2) / adf_result[0]

# Initialize position and cash balance
position = 0
cash = 100000

# Define trading function
def trade(date, price):
    global position, cash
    if position == 0 and price < aapl.loc[date, 'Lower']:
        # Enter long position if price falls below lower band
        position = cash / price
        cash = 0
    elif position > 0 and price > aapl.loc[date, 'SMA']:
        # Exit long position if price rises back above SMA
        cash = position * price
        position = 0

# Loop over time series and perform trades
for date, row in aapl.iterrows():
    if date >= aapl.index[0] + pd.Timedelta(days=window):
        # Compute p-value of ADF test for the past 20 days
        series = aapl.loc[date-pd.Timedelta(days=window):date, 'Adj Close']
        pvalue = adf_test(series)

        # Perform trade if p-value is below 0.05
        if pvalue < 0.05:
            trade(date, row['Adj Close'])


[*********************100%***********************]  1 of 1 completed


Buying a stock when its short-term moving average crosses above its long-term moving average and selling when the short-term moving average crosses below the long-term moving average.

import backtrader as bt
import alpaca_backtrader_api as alpaca
class MovingAverageCross(bt.Strategy):
    params = (
        ('short_term_period', 10),
        ('long_term_period', 50)
    )
    def __init__(self):
        # Calculate the short-term and long-term moving averages
        self.short_term_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_term_period
        )
        self.long_term_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_term_period
        )
    def next(self):
        # Check if the short-term MA crosses above the long-term MA
        if self.short_term_ma[0] > self.long_term_ma[0] and \
                self.short_term_ma[-1] <= self.long_term_ma[-1]:
            # Buy the stock
            self.buy()
        # Check if the short-term MA crosses below the long-term MA
        elif self.short_term_ma[0] < self.long_term_ma[0] and \
                self.short_term_ma[-1] >= self.long_term_ma[-1]:
            # Sell the stock
            self.sell()
cerebro = bt.Cerebro()

store = alpaca.AlpacaStore(
    key_id='<your-api-key-id>',
    secret_key='<your-api-secret-key>',
    paper=True
)
DataFactory = store.getdata
data = DataFactory(dataname='AAPL', historical=True, fromdate=datetime(2022, 1, 1))
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(MovingAverageCross)

# Set the commission and slippage
cerebro.broker.setcommission(commission=0.001)
cerebro.broker.set_slippage_fixed(0.01, slip_open=True, slip_match=True, slip_out=True)

# Set the initial capital
cerebro.broker.setcash(10000.0)

# Run the backtest
cerebro.run()

# Print the final portfolio value
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Buys bonds with high yields and sells bonds with low yields

In [ ]:
import backtrader as bt

class BondYield(bt.feeds.GenericCSVData):
    params = (
        ('nullvalue', float('NaN')),
        ('dtformat', ('%Y-%m-%d')),
        ('datetime', 0),
        ('high', 1),
        ('low', 2),
        ('open', 3),
        ('close', 4),
        ('volume', None),
        ('openinterest', None),
    )

    def __init__(self):
        bonds = {
            '10Y': 'https://datahub.io/core/bond-yields-us-10y/r/monthly.csv',
            '2Y': 'https://datahub.io/core/bond-yields-us-2y/r/monthly.csv',
            '30Y': 'https://datahub.io/core/bond-yields-us-30y/r/monthly.csv',
            '5Y': 'https://datahub.io/core/bond-yields-us-5y/r/monthly.csv'
        }
        super().__init__(dataname=bonds)

class BondYieldStrategy(bt.Strategy):
    def __init__(self):
        self.bonds = self.datas

    def next(self):
        sorted_bonds = sorted(self.bonds, key=lambda b: b.close[0])
        long_bond = sorted_bonds[0]
        short_bond = sorted_bonds[-1]
        self.sell(short_bond)
        self.buy(long_bond)

cerebro = bt.Cerebro()

# Add the bonds to the cerebro instance
for bond in BondYield().bonds.values():
    cerebro.adddata(bond)

# Add the strategy
cerebro.addstrategy(BondYieldStrategy)

# Run the backtest
cerebro.run()


Longs stocks expected to outperform and shorts stocks expected to underperform, creating a market-neutral portfolio.

In [24]:
import pandas as pd
import numpy as np

# Load stock data
df = pd.read_csv('stock_data.csv', index_col=0)

# Calculate expected returns using historical data
returns = df.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Calculate expected returns for long and short portfolios
long_portfolio_returns = mean_returns.values
short_portfolio_returns = -mean_returns.values

# Calculate portfolio weights for long and short portfolios
long_weights = np.dot(inv_cov_matrix, long_portfolio_returns) / np.sum(np.dot(inv_cov_matrix, long_portfolio_returns))
short_weights = np.dot(inv_cov_matrix, short_portfolio_returns) / np.sum(np.dot(inv_cov_matrix, short_portfolio_returns))

# Create long-short portfolio
portfolio_weights = long_weights - short_weights

# Calculate portfolio returns
portfolio_returns = np.dot(portfolio_weights, returns.T)


FileNotFoundError: [Errno 2] No such file or directory: 'stock_data.csv'